This is a personal workbook used to process my own data but has been made public for the use of others. If you find any errors or need further help using this workbook please contact k8macarthur@gmail.com.

Begin by importing the Hyperspy Function Library.

In [1]:
import matplotlib
matplotlib.rcParams["backend"] = 'Qt4Agg'
import hyperspy.api as hs
import numpy as np
%matplotlib qt4

# Step 1: Import the Pt Wedge

Opens a load window. Use hs.load('filename') if filename is known specifically.

In [2]:
Pt_wedge = hs.load()

These next two lines are needed for .bcf files from Bruker/Espirit as the imported data comes as a list containing both the ADF image and EDX spectral data cube.

In [3]:
Pt_wedge_image = Pt_wedge[0]
Pt_wedge = Pt_wedge[1]

TypeError: 'EDSTEMSpectrum' object does not support indexing

In [4]:
Pt_wedge.set_signal_type('EDS_TEM')
Pt_wedge.metadata.General.title = 'Original Data Wedge'

Check that the wedge has sensible dimensions and shape, 
that the labelling has been done correctly.

In [6]:
Pt_wedge

<EDSTEMSpectrum, title: Original Data Wedge, dimensions: (1024, 64|1024)>

Adjust the metadata for all the microscope parameters.

In [7]:
Pt_wedge.axes_manager[0].name = 'x'
Pt_wedge.axes_manager[1].name = 'y'
Pt_wedge.axes_manager

<Axes manager, axes: (1024, 64|1024)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |   1024 |      0 |       0 |      18 |     nm 
               y |     64 |      0 | 9.7e+03 |      18 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
          Energy |   1024 |        |   -0.23 |    0.01 |    keV

The axes_manager.gui() can be used to set all the axis properties. The scale of the navigational axis should be in nm for the later processing.

In [8]:
Pt_wedge.axes_manager.gui()

The next two lines can be used if the data needs cropping before further analyis. 

Data is cropped based on the pixel number ranges used. The crop range can be plotted first, before cropping.

In [10]:
Pt_wedge.inav[0:1024, 553:617].plot()

Cropping is performed and then the new data replotted.

In [11]:
Pt_wedge0 = Pt_wedge
Pt_wedge = Pt_wedge.inav[0:1024, 553:617]
Pt_wedge.plot()

In [8]:
Pt_wedge.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 45.0
│       │       ├── detector_type = SuperX
│       │       ├── elevation_angle = 18.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 1
│       │       └── real_time = 0.002696
│       ├── beam_current = 0.039
│       ├── beam_energy = 200.0
│       ├── stage_x = None
│       ├── stage_y = None
│       └── tilt_stage = 0.0
├── General
│   ├── original_filename = Pt01-compressed.bcf
│   └── title = Original Data Wedge
├── Sample
│   ├── elements = ['Pt']
│   ├── name = Image 1975
│   └── xray_lines = ['Pt_La']
└── Signal
    ├── binned = True
    └── signal_type = EDS_TEM

Adjust all the metadata for the microscope parameters.

(Live_time is not needed for quantification but currently prevents the code from running further if not set.)

In [13]:
#Pt_wedge.set_microscope_parameters(beam_energy = 200)
Pt_wedge.set_microscope_parameters(live_time = 1)
Pt_wedge.set_microscope_parameters(beam_current = 0.039) #in nA
Pt_wedge.set_microscope_parameters(real_time = 0.002696) #dwell time per pixel

Adjust the metadata for all the sample parameters.

In [9]:
Pt_wedge.set_elements(['Pt'])
Pt_wedge.set_lines(['Pt_La'])

Finally look at the original raw data in plot mode.

In [10]:
Pt_wedge.plot(True)

Save the data, with all the added metadata, for repeat processing if necessary.

In [17]:
Pt_wedge.save('Pt01EDX-processed')

# Step 2: Extract the intensity map for the wedge

Define the background subtraction windows:

*These can be edited or modified manually as necessary depending on what they look like.*

In [11]:
Pt_wedge.add_lines()
bw_Pt = Pt_wedge.estimate_background_windows(line_width=[5.0, 2.0])
bw_Pt

array([[ 8.47913271,  8.63962726,  9.7630891 ,  9.92358364]])

Define the integration windows:

*The automatic integration window is 2xFWHM. However this might vary depending on counts. For accurate quantification we want to integrate over the same absolute windows so we need to define them for exact use later on. Using FWHM is a reasable first measure approach for this.*

In [12]:
iw_Pt =  Pt_wedge.estimate_integration_windows(windows_width=2)
iw_Pt[0][1] = 9.65
iw_Pt

[[9.281605451805987, 9.65]]

Plot the sum spectrum with all the background windows to check that nothing overlaps detrimentally. 

*The integration windows should only contain the individual element peak in question otherwise some form of deconvolution method may be required for integration. The background windows should overlap with no peaks.*

In [13]:
Pt_wedge.sum().plot(True, background_windows=bw_Pt, integration_windows=iw_Pt)

//anaconda/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Now extract the intensity map from your spectrum image.

In [46]:
inten_Pt = Pt_wedge.get_lines_intensity(integration_windows=iw_Pt, background_windows=bw_Pt)
inten_Pt[0].plot()
inten_Pt

[<Signal2D, title: X-ray line intensity of Original Data Wedge: Pt_La at 9.44 keV, dimensions: (|1024, 64)>]

This final line is needed when combining multiple x-ray lines for each element.

In [17]:
inten_Pt = inten_Pt[0]+inten_Pt[1]

IndexError: list index out of range

# Step 3: Repeat Everything Above for Ni

In [16]:
Ni_wedge = hs.load()
#Separate out the image and spectrum image from .bcl files.
#Ni_wedge_image = Ni_wedge[0]
#Ni_wedge = Ni_wedge[1]

Ni_wedge.set_signal_type('EDS_TEM')
Ni_wedge.metadata.General.title = 'Original Data Wedge'

Ni_wedge

<EDSTEMSpectrum, title: Original Data Wedge, dimensions: (1024, 64|1024)>

In [17]:
Ni_wedge.axes_manager

<Axes manager, axes: (1024, 64|1024)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
           width |   1024 |      0 |       0 |      18 |     nm 
          height |     64 |      0 | 8.4e+03 |      18 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
          Energy |   1024 |        |   -0.23 |    0.01 |    keV

In [24]:
Ni_wedge.axes_manager.gui()

In [105]:
Ni_wedge.set_elements(['Ni'])
Ni_wedge.set_lines(['Ni_Ka', 'Ni_Kb'])
Ni_wedge.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 45.0
│       │       ├── detector_type = SuperX
│       │       ├── elevation_angle = 18.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── live_time = 1
│       │       └── real_time = 0.0025669999999999998
│       ├── beam_current = 0.073999999999999996
│       ├── beam_energy  = 200
│       ├── stage_x = None
│       ├── stage_y = None
│       └── tilt_stage = 0.0
├── General
│   ├── original_filename = Ni02-compressed.bcf
│   └── title = Original Data Wedge
├── Sample
│   ├── elements = ['Ni']
│   ├── name = Image 1978
│   └── xray_lines = ['Ni_Ka', 'Ni_Kb']
└── Signal
    ├── binned = True
    └── signal_type = EDS_TEM

In [106]:
Ni_wedge.set_microscope_parameters(beam_energy = 200)
Ni_wedge.set_microscope_parameters(live_time = 1)
Ni_wedge.set_microscope_parameters(beam_current = 0.074) #in nA
Ni_wedge.set_microscope_parameters(real_time = 0.002567) #dwell time per pixel

In [107]:
Ni_wedge.plot(True)
Ni_wedge.add_lines()
bw_Ni = Ni_wedge.estimate_background_windows(line_width=[5.0, 2.0])
bw_Ni

array([[ 6.61175942,  6.75614952,  8.5667927 ,  8.71783905],
       [ 6.61175942,  6.75614952,  8.5667927 ,  8.71783905]])

In [108]:
iw_Ni = Ni_wedge.estimate_integration_windows(windows_width=2)
iw_Ni

[[7.333709903386694, 7.622490096613307], [8.11365364949791, 8.415746350502088]]

In [20]:
Ni_wedge.sum().plot(True, background_windows=bw_Ni, integration_windows=iw_Ni)

//anaconda/lib/python3.5/site-packages/matplotlib/__init__.py:892: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [32]:
Ni_wedge.inav[0:1024, 480:544].plot()

In [33]:
Ni_wedge0 = Ni_wedge
Ni_wedge = Ni_wedge.inav[0:1024, 480:544]
Ni_wedge.plot()

In [34]:
Ni_wedge.save('Ni02EDX-processed')

In [22]:
inten_Ni = Ni_wedge.get_lines_intensity(background_windows=bw_Ni)
inten_Ni

[<Signal2D, title: X-ray line intensity of Original Data Wedge: Ni_Ka at 7.48 keV, dimensions: (|1024, 64)>]

In [ ]:
inten_Ni = inten_Ni[0]+inten_Ni[1]

# Step 4: Convert from the intensity map to Cross Section
The following function will calculate the cross section of a for a single element wedge sample with a known angle. It assumes that the line profile is parallel to the x-axis in the image. 

*The function will also print the error in the gradient measurement which is important for subsequent error analysis of the quantificationt results.*

In [25]:
def cross_section_from_wedge(intensity,
                            angle,
                            atomic_density):
    """
    
    Parameters
    ----------
        
    intensity : array
        
    angle : float
        Of the FIB wedge in degrees.
    atomic_density: float 
        Of the particular element in units of atoms/nm^3.
        (Pt = 33.2026, Ni = 45.8566)
    
    Returns
    -------
    cross_section : float
        partial ionisation cross section
    
    """
    
    from scipy import constants
    import math
    e = constants.e
    parameters = intensity[0].metadata.Acquisition_instrument.TEM
    curr = parameters.beam_current
    dwell = parameters.Detector.EDS.real_time
    width = intensity[0].axes_manager[0].scale
    angle = angle/360*constants.pi*2

    #begin by converting the intensity array to a line profile.
    #works for square images, needs checking for those which are non-square.
    line_scan = np.zeros(intensity[0].axes_manager.signal_shape[0])
    line_scan.astype(float)
    for row in range(intensity[0].axes_manager.signal_shape[1]):
        for col in range(intensity[0].axes_manager.signal_shape[0]):
            line_scan[col] = intensity[0].data[row, col] + line_scan[col]
    
    line_scan = line_scan/(intensity[0].axes_manager.signal_shape[1]*width)
    x_data = np.zeros(intensity[0].axes_manager.signal_shape[0])
    x_data.astype(float)
    for i in range(intensity[0].axes_manager.signal_shape[0]):
        x_data[i] = i*width
    
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 6), dpi=80)
    plt.plot(x_data, line_scan)
    plt.show()
    #curve fitting to line scan to find gradient
    #using  a non-linear least squares fitting.
    from scipy.optimize import curve_fit
    def func(x, m, c):
        return m*x + c
    popt, pcov = curve_fit(func, x_data, line_scan)
    
    m = popt[0]
    print('m =', m)
    perr = np.sqrt(np.diag(pcov))
    m_err = perr[0] #the error in m measurement
    print('Error in m =', m_err)
    
    #calculate cross_section
    cross_section = (e*1E19*m)/(curr*dwell*atomic_density*math.tan(angle))
    return cross_section

Calculate the cross section for each element from the previously extracted intensity maps.

In [84]:
inten_Pt[0] = inten_Pt[0].isig[:475, :]
inten_Pt

[<Signal2D, title: X-ray line intensity of Original Data Wedge: Pt_La at 9.44 keV, dimensions: (|475, 64)>]

In [97]:
inten_Pt[0].save('PtEDX01-inten')

In [85]:
cross_section_Pt = cross_section_from_wedge(inten_Pt,
                                            0.269,
                                            33.2026)

m = 7.99456305256e-05
Error in m = 8.29810009806e-07


//anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


An interative process of cropping the line profile to only the linear region is needed for accurate cross section determination. We only want the gradient where the x-ray generation is linear with sample thickness.

In [80]:
inten_Ni[0] = inten_Ni[0].isig[:400, :]
inten_Ni

[<Signal2D, title: X-ray line intensity of Original Data Wedge: Ni_Ka at 7.48 keV, dimensions: (|400, 64)>]

In [81]:
cross_section_Ni = cross_section_from_wedge(inten_Ni,
                                           0.401,
                                           45.8566)

m = 0.000323058447471
Error in m = 2.00357066389e-06


//anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [86]:
print('Cross Section Pt', cross_section_Pt)
print('Cross Section Ni', cross_section_Ni)

Cross Section Pt 7.81476090922
Cross Section Ni 8.48991263031


In [98]:
inten_Ni[0].save('NiEDX02-inten')

# Finally, store the cross sections and windows into a data base for later use:

Convert the cross section and windows into a dictionary format.

In [100]:
calibration_Pt = dict = {'lines': 'PLa', 'cross_section': cross_section_Pt,
                         'Background windows': bw_Pt, 'Integration windows': iw_Pt}
calibration_Ni = dict = {'lines': 'CoKa', 'cross_section': cross_section_Ni, 
                         'Background windows': bw_Ni, 'Integration windows': iw_Ni}

In [101]:
calibration_Pt

{'Background windows': array([[ 8.47913271,  8.63962726,  9.7630891 ,  9.92358364]]),
 'Integration windows': [[9.281605451805987, 9.65]],
 'cross_section': 7.8147609092183261,
 'lines': 'PLa'}

In [102]:
calibration = dict = {'Ni': calibration_Ni, 'Pt': calibration_Pt}
calibration['Ni']['cross_section']

8.4899126303055183

In [103]:
cal = hs.signals.BaseSignal([0])
cal.metadata.calibration = calibration
cal.save('ChemiSTEMCal_V01')

Overwrite 'ChemiSTEMCal_V01.hdf5' (y/n)?
y


The following function can be used to check which elements are stored in the dictionary.

In [104]:
calibration.keys()

dict_keys(['Ni', 'Pt'])

Any functions below this point are for testing, and not needed in the method above.

In [92]:
calibration['Co'] = calibration_Ni

In [93]:
del calibration['Co']

In [94]:
calibration2 = cal.metadata.calibration